<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Computational Optimal Transport</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>. Based on a project at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/compopt</a>.

# Style Transfer in Images

We use in this notebook some basic applications of Optimal Transport for manipulating images.

The common background is the following.

We consider a discrete measures $\alpha$ defined as a pair of vectors of $\mathbb{R}^n$, where $a$ is the vector of weights and $x$ is the vector of locations (e.g., points in $\mathbb{R}^q$); the discrete measure is

$$
    \alpha = \sum_{i=1}^n a_i \delta_{x_i}   
$$

In addition, to require the measure to be a probability, we require that $\sum_{i=1}^n a_i = 1$.

Similary, we can define a second discrete measure $\beta$, with vectors $b$ and $y$, such that:

$$
    \beta = \sum_{i=1}^m b_i \delta_{y_i}   
$$

(and $\sum_{j=1}^m b_j = 1$)

We define the Optimal Transport problem between $\alpha$ and $\beta$ as the following Linear Programming (LP) problem:

$$
\begin{align}
OT_p(\alpha, \beta) := \min \quad & \sum_{i=1}^n \sum_{j=1}^m |x_i - y_j|_p \pi_{ij} \\
\text{s.t.} \quad & \sum_{j=1}^m \pi_{ij} = a_i & i = 1,\dots, n \\
& \sum_{i=1}^n \pi_{ij} = b_j & j = 1,\dots, m \\
& \pi_{ij} \geq 0
\end{align}
$$

By standard LP argument, this problem admits the following dual LP problem, in variables $f_i$ (for marginal constraints defined on $a_i$) and $g_j$ (for marginal constraints defind on $b_j$):

$$
\begin{align}
D_p(\alpha, \beta) := \max \quad & \sum_{i=1}^n a_i f_i + \sum_{j=1}^m b_j g_j \\
\text{s.t.} \quad & f_i + g_j \leq |x_i - y_j|_p & i=1,\dots,n \quad j=1,\dots,m \\
& f_i, g_j \text{ free in sign}
\end{align}
$$

The Wasserstein distance of ordeer $p$ between two (discrete) measure is defined as
$$
W_p(\alpha, \beta) := \left(OT_p(\alpha, \beta)\right)^{\frac{1}{p}}
$$
and defines a distance among discrete measures.

### Applications with images
In applying optimal transport to images, we need to map an image to a discrete measure, defining the vector of weights (eventually all equal) and a vector of locations (e.g., points in $\mathbb{R}^q$).

In the first example, we will use points on $\mathbb{R}$ (that is, 1-dimensional optimal transport), and in the second example, we will use points on $\mathbb{R}^3$. In both cases, we will start by using vector of weights all equal among them.

## Example 1: Histogram Transfer for black-and-white images

Given a black and white image of size $N \times N$, it is possible to flatten all the pixels into a single vector of dimension $N^2$, where each element is equal to the pixel intensity (i.e., a value between 0 and 255, which is normalized into a value in $[0 \dots 1]$). We can consider this vector as a discrete measure $\alpha$, where each weight $a_i = \frac{1}{n}$, and each location $x_i$ is the pixel intensity.
This way we build a histogram of color intensity as follows.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def ReadJpegBW(filename, xoff=0, yoff=0):
    A = plt.imread(filename)
    # Hack for resizing the image in 200x200 (center on Monge face)
    A = A[xoff:200+xoff, yoff:200+yoff, 0].astype(float)/255
    return A

In [ ]:
def ShowImage(A):
    fig, ax = plt.subplots()
    plt.imshow(A, cmap='gray')
    ax.autoscale()
    ax.set_aspect('equal', 'box')
    plt.axis('off')
    plt.show()

In [ ]:
def PlotHistogram(A):
    A = A.flatten()
    I = np.argsort(A)
    plt.hist(A[I], bins=50, color='red', alpha=0.5, density=True)
    plt.show()

In [ ]:
# Metti il comando wget per prendere l'immagine
A = ReadJpegBW('../data/foto1.jpeg')
ShowImage(A)

In [ ]:
PlotHistogram(A)

**EXERCISE 1**: Read a second image (a Monge portrait), and solve the problem of *transfering* the distribution of pixel intensities from the first image to the second one. Use the complete distributions of pixel intensity of two image with the same size of 200x200 pixels. 
Plot the new image and the corresponding histogram if pixel intensities.

In [ ]:
B = ReadJpegBW('../data/monge.jpeg', xoff=75, yoff=50)
ShowImage(B)
PlotHistogram(B)

**CHALLENGE 1**: Can you find the optimal histogram transfer between the two images without cropping, that is, between two images having a different number of pixels? Which algorithm would you use?

## Example 2: Color Transfer of color images
In this second application, we deal with color images, that is, with tensors of dimension $m \times n \times 3$, and we want to find the optimal map from the color palette of an image into the other.

First, we show a few basic functions to deal with color images, and we samples of points in $\mathbb{R}^3$.

In [ ]:
def LoadImage(filename):
    A = plt.imread(filename).astype(np.float64)/255
    return A

In [ ]:
from random import sample, seed
seed(13)
def DisplayCloud(A, samples=100):
    n, m, l = A.shape
    print('shape', n, m, l)
    C = A.reshape(n*m, 3)
    s = sample(range(0, m*n), samples)
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    plt.scatter(x=C[s, 0], y=C[s, 1], zs=C[s, 2], s=100, c=C[s])
    plt.show()

In [ ]:
A = LoadImage('../data/ferrari.jpg')
B = LoadImage('../data/fioreblu.jpeg')

In [ ]:
ShowImage(A)

In [ ]:
ShowImage(B)

In [ ]:
DisplayCloud(A, samples=1000)

In [ ]:
DisplayCloud(B)

**EXERCISE 2**: Write the optimal transport problem to map a sample of points from the first image to the second, in order to minimize the overall distance of the mapping. Begin with using a small number of sample points, then try to study the performarnce of the OT tranposrt algorithm as a function of the number of sample points.

**CHALLENGE 2**: Can you find a method and/or model to tranposrt exactly every pixel of the first image into the second (eventually by splitting masses)?